# 模型训练

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# set gpu for training
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

## 1. 加载数据集

In [ ]:
X, y = np.load('./Data/X.npy'), np.load('./Data/y.npy')

## 2. 拆分数据集
*数据增强

In [ ]:
X, y = np.expand_dims(X, -1), np.array(y)
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = X[:5000,:,:,:], X[5000:,:,:,:], y[:5000], y[5000:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## 3. 设计模型

In [ ]:
model = tf.keras.Sequential(name='LeNet5')
model.add(tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), strides=(2, 2), input_shape=(512, 512, 1), activation='relu'))
model.add(tf.keras.layers.MaxPool2D()) # 最大池化层
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), strides=(2, 2), activation='relu')) # 卷积层
model.add(tf.keras.layers.MaxPool2D()) # 最大池化层
model.add(tf.keras.layers.Flatten()) # 展平层
model.add(tf.keras.layers.Dense(units=120, activation='relu')) # units: 全连接层的神经元数量
model.add(tf.keras.layers.Dense(units=84, activation='relu'))
model.add(tf.keras.layers.Dense(units=32, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
model.summary()

## 4. 编译模型

In [ ]:
#对模型进行编译
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.binary_crossentropy, 
    metrics=tf.keras.metrics.binary_accuracy
)

## 5. 回调函数

In [ ]:
# 回调函数
model_check_point = tf.keras.callbacks.ModelCheckpoint(
    filepath='./Models/best_model_5000_32_128.h5', 
    monitor='val_binary_accuracy', 
    verbose=1, 
    save_best_only=True, # 仅保存最优模型
    save_weights_only=False, # 是否仅保存权重
    mode='auto' # 设置检测参数的最优方向
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    min_delta=0, # 触发终止的阈值: 最小增量
    patience=10, # 等待训练停止的 无改善时期 的数量
    verbose=1,
    mode='auto'
)

# 设置权重衰减
def scheduler(epoch):
    if epoch % 5 == 0 and epoch != 0:
        lr = tf.keras.backend.get_value(model.optimizer.lr)
        tf.keras.backend.set_value(model.optimizer.lr, lr*0.9)
        print('lr changed to {}'.format(lr*0.9))
    return tf.keras.backend.get_value(model.optimizer.lr)
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

# 保存训练日志
csv_logger = tf.keras.callbacks.CSVLogger('./Logs/training_5000_32_128.log', separator=',', append=False)
callbacks = [model_check_point, early_stopping, learning_rate_scheduler, csv_logger]

## 6. 模型的训练

In [ ]:
#训练模型
history = model.fit(
    X_train, 
    y_train, 
    epochs=32, # 批次量
    batch_size=128, # 批量大小
    validation_split=0.2, # 验证集比例
    callbacks=callbacks # 回调函数
)

## 7. 模型的评估

In [ ]:
#进行模型的评估
model.evaluate(X_test, y_test)

## 8. 模型的预测可视化

In [ ]:
best_model = tf.keras.models.load_model('./Models/best_model_5000_20_128.h5')

In [ ]:
y_pred = best_model.predict(X_test[2:10,:,:,:])
y_pred = [1 if i > 0.5 else 0 for i in y_pred]
y_pred, y_test[2:10]

In [ ]:
label_dict = {0: 'Not Sick', 1: 'Sick'}

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(len(y_pred)):
    ax = plt.subplot(4,4,i+1)
    ax.axis('off')
    ax.set_title(f'y_true={y_test[2+i]}\ny_pred={y_pred[i]}')
    ax.imshow(X_test[2+i,:,:,:].squeeze())
plt.show()

In [ ]:
plt.plot(range(len(history.history['loss'])), history.history['loss'])
plt.plot(range(len(history.history['val_loss'])), history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
plt.plot(range(len(history.history['binary_accuracy'])), history.history['binary_accuracy'])
plt.plot(range(len(history.history['val_binary_accuracy'])), history.history['val_binary_accuracy'])
plt.legend(['binary_accuracy', 'val_binary_accuracy'])
plt.show()